# CORRIEDALE_INIA_UY
Describing `CORRIEDALE_INIA_UY.zip` file

In [1]:
import io
import itertools

from zipfile import ZipFile
from pathlib import Path

import src.features.illumina

In [2]:
project_dir = Path.cwd().parents[1]
datafile = project_dir / "data/raw/background/CORRIEDALE_INIA_UY.zip"

Esploring Dataset and try to get coordinate version

In [3]:
handle = ZipFile(datafile)
handle.printdir()

File Name                                             Modified             Size
CORRIEDALE_UY_60_INIA_Ovine_14sep2010.ped      2020-12-09 17:21:00     13019391
CORRIEDALE_UY_60_INIA_Ovine_14sep2010.map      2020-12-09 17:21:04      1502260
INIA_Ovine_14sep2010_FinalReport.txt           2020-12-03 14:13:08    159645949
INIA_Ovine_SNP_Map_14sep2010.txt               2020-12-03 14:13:28      3012148


In [4]:
with io.TextIOWrapper(handle.open("INIA_Ovine_14sep2010_FinalReport.txt"), encoding="utf-8") as f:
    for line in itertools.islice(f, 11):
        print(line.strip())

[Header]
GSGT Version	1.6.3
Processing Date	9/15/2010 10:56 AM
Content		OvineSNP50_B.bpm
Num SNPs	54241
Total SNPs	54241
Num Samples	60
Total Samples	60
[Data]
SNP Name	Sample ID	Allele1 - Forward	Allele2 - Forward	Allele1 - Top	Allele2 - Top	Allele2 - AB	Allele1 - AB	GC Score	X	Y
250506CS3900065000002_1238.1	1	C	C	G	G	B	B	0.9239	0.019	0.949


In [5]:
data_coordinates = dict()
with io.TextIOWrapper(handle.open("CORRIEDALE_UY_60_INIA_Ovine_14sep2010.map"), encoding="utf-8") as f:
    for record in f:
        record = record.split()
        data_coordinates[record[1]] = (record[0], int(record[3]))

Seems to be the old (pre oarv3) system. Check with the oldest chip I have

In [6]:
chip_dir = project_dir / "data/external/SHE/ILLUMINA/"
old_chip3_file = chip_dir / "ovinesnp50_b.csv"
old_chip3 = dict()
for record in src.features.illumina.read_snpChip(old_chip3_file):
    old_chip3[record.name] = (record.chr, record.mapinfo)

In [7]:
count = 0
missing = 0

for key, value in old_chip3.items():
    if not key in data_coordinates:
        missing += 1
        continue
        
    if value != data_coordinates[key]:
        count += 1
        print(key, value, data_coordinates[key])
        
print(f"\nN of SNPs in different positions in coordiedale and old chip3: {count}")
print(f"\nN of missing SNPs in corriedale: {missing}")

s17862.1 ('Contig', 0) ('CONTIG', 0)

N of SNPs in different positions in coordiedale and old chip3: 1

N of missing SNPs in corriedale: 0


This dataset seems to have all the 50K SNP in the old reference system